In [2]:
# https://machinelearningmastery.com/pytorch-tutorial-develop-deep-learning-models/
# https://colab.research.google.com/github/rpi-techfundamentals/website_spring_2020/blob/master/content/notebooks/20-deep-learning1/06-regression-bh-pytorch.ipynb#scrollTo=xD9PhAU7hoqT
#!pip install torchvision
import numpy as np
import pandas as pd
import pyreadr
import scipy
#Define the model 
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils import shuffle
from torch.autograd import Variable


In [3]:
TrainSet_eonr2_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/TrainSet_eonr2.rds")[None] # also works for RData


In [4]:
y_train = TrainSet_eonr2_df['eonr']
X_train = TrainSet_eonr2_df.drop('eonr', axis=1)


In [5]:
#Define training hyperprameters.
batch_size = 50
num_epochs = 200
learning_rate = 0.01
size_hidden= 100

#Calculate some other hyperparameters based on data.  
batch_no = len(X_train) // batch_size  #batches
cols=X_train.shape[1] #Number of columns in input matrix
n_output=1

In [6]:
#Create the model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assume that we are on a CUDA machine, then this should print a CUDA device:
print("Executing the model on :",device)
class Net(torch.nn.Module):
    def __init__(self, n_feature, size_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(cols, size_hidden)   # hidden layer
        self.predict = torch.nn.Linear(size_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.predict(x)             # linear output
        return x
net = Net(cols, size_hidden, n_output)

Executing the model on : cuda:0


In [7]:
#Adam is a specific flavor of gradient decent which is typically better
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.2)
criterion = torch.nn.MSELoss(size_average=False)  # this is for regression mean squared loss

/opt/anaconda3/envs/GEOANN/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [8]:
X_train=X_train.values
y_train=y_train.values

In [9]:
running_loss = 0.0
for epoch in range(num_epochs):
    #Shuffle just mixes up the dataset between epocs
    X_train, y_train = shuffle(X_train, y_train)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.FloatTensor(y_train[start:end]))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        #print("outputs",outputs)
        #print("outputs",outputs,outputs.shape,"labels",labels, labels.shape)
        loss = criterion(outputs, torch.unsqueeze(labels,dim=1))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('Epoch {}'.format(epoch+1), "loss: ",running_loss)
    running_loss = 0.0


Epoch 1 loss:  206362349.484375
Epoch 2 loss:  14107508.875
Epoch 3 loss:  14251030.5078125
Epoch 4 loss:  13224424.59375
Epoch 5 loss:  12918470.1796875
Epoch 6 loss:  12735272.296875
Epoch 7 loss:  12384012.625
Epoch 8 loss:  12402268.125
Epoch 9 loss:  11995507.4765625
Epoch 10 loss:  12832892.515625
Epoch 11 loss:  12172042.0390625
Epoch 12 loss:  11587917.1328125
Epoch 13 loss:  11952702.4765625
Epoch 14 loss:  11989866.7890625
Epoch 15 loss:  11689623.765625
Epoch 16 loss:  11458095.84375
Epoch 17 loss:  11945276.5078125
Epoch 18 loss:  11959250.98046875
Epoch 19 loss:  10982238.859375
Epoch 20 loss:  11162222.6484375
Epoch 21 loss:  10989309.6015625
Epoch 22 loss:  10527156.82421875
Epoch 23 loss:  10350854.6484375
Epoch 24 loss:  11070669.7421875
Epoch 25 loss:  10669662.7890625
Epoch 26 loss:  10280334.17578125
Epoch 27 loss:  10326266.09375
Epoch 28 loss:  10691566.1875
Epoch 29 loss:  10157478.94140625
Epoch 30 loss:  10739198.828125
Epoch 31 loss:  10023945.88671875
Epoch 3

In [10]:
torch.save(net.state_dict(), "/home/germanm2/n_policy_box/Data/files_rds/cnn_model.pth")

In [11]:
#import pandas as pd
X = Variable(torch.FloatTensor(X_train)) 
result = net(X)
# pred=result.data[:,0].numpy()
# print(len(pred),len(y_train))
# r2_score(pred,y_train)

In [12]:
prediction_set_aggregated_df = pyreadr.read_r("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_dt.rds")[None] # also works for RData
prediction_set_aggregated_df.head()

,id_10,id_field,region,z,rain_30,rain_60,rain_90,t_max_30,t_max_60,t_max_90,...,day_v5,lai_v5,whc,oc_20cm_v5,sw_dep_v5,n_0_60cm_v5,surfaceom_wt_v5,sand_40cm,clay_40cm,area_ha
0,5,3.0,3.0,11,125.0,229.0,17.0,22.050000,15.216667,8.250000,...,152.0,0.408,294.075688,1.620588,629.483784,35.048204,465.378891,5.071375,21.024234,40.0
1,5,3.0,3.0,13,140.0,108.0,19.0,22.133333,15.933333,3.383333,...,147.0,0.402,294.075688,1.625338,666.127141,42.310082,511.010443,5.071375,21.024234,40.0
2,5,3.0,3.0,15,96.0,101.0,39.0,20.666667,16.783333,9.250000,...,157.0,0.406,294.075688,1.622531,620.573678,46.126236,497.375201,5.071375,21.024234,40.0
3,5,3.0,3.0,17,96.0,47.0,33.0,22.016667,17.066667,9.600000,...,157.0,0.397,294.075688,1.620151,629.544435,70.271481,462.792524,5.071375,21.024234,40.0
4,5,3.0,3.0,19,43.0,183.0,51.0,23.916667,12.283333,8.650000,...,147.0,0.398,294.075688,1.623718,623.843297,47.598318,721.119533,5.071375,21.024234,40.0


In [13]:
X_pred = prediction_set_aggregated_df[['rain_30', 'rain_60','rain_90', 't_max_30', 't_max_60', 't_max_90', 't_min_30', 't_min_60', 't_min_90', 'Y_prev',
 'Y_corn_lt_avg', 'day_sow', 'day_v5', 'lai_v5', 'whc', 'oc_20cm_v5', 'sw_dep_v5', 'n_0_60cm_v5', 'surfaceom_wt_v5', 'sand_40cm', 'clay_40cm']]
X_pred=X_pred.values
X_pred

array([[125.        , 229.        ,  17.        , ..., 465.37889136,
          5.07137482,  21.02423362],
       [140.        , 108.        ,  19.        , ..., 511.01044335,
          5.07137482,  21.02423362],
       [ 96.        , 101.        ,  39.        , ..., 497.37520121,
          5.07137482,  21.02423362],
       ...,
       [141.        , 124.        , 120.        , ..., 211.43236352,
         20.46973879,  19.59067624],
       [190.        ,  16.        , 132.        , ...,  94.55417258,
         20.46973879,  19.59067624],
       [ 94.        , 143.        , 214.        , ..., 142.36507711,
         20.46973879,  19.59067624]])

In [16]:
# MAKE PREDICTIONS
X = Variable(torch.FloatTensor(X_pred)) 
y_pred = net(X) #This outputs the value for regression
y_pred=y_pred.data[:,0].numpy()
y_pred

array([136.59433, 181.84709, 179.36964, ..., 177.03264, 257.33984,
       216.06119], dtype=float32)

In [17]:
prediction_set_aggregated_df['eonr_pred'] = y_pred
prediction_set_aggregated_df.head

<bound method NDFrame.head of        id_10  id_field  region   z  rain_30  rain_60  rain_90   t_max_30  \
0          5       3.0     3.0  11    125.0    229.0     17.0  22.050000   
1          5       3.0     3.0  13    140.0    108.0     19.0  22.133333   
2          5       3.0     3.0  15     96.0    101.0     39.0  20.666667   
3          5       3.0     3.0  17     96.0     47.0     33.0  22.016667   
4          5       3.0     3.0  19     43.0    183.0     51.0  23.916667   
...      ...       ...     ...  ..      ...      ...      ...        ...   
60362   1526       1.0     1.0  29    251.0    117.0     53.0  23.083333   
60363   1526       1.0     1.0   3    138.0     79.0    107.0  22.150000   
60364   1526       1.0     1.0   5    141.0    124.0    120.0  21.766667   
60365   1526       1.0     1.0   7    190.0     16.0    132.0  21.750000   
60366   1526       1.0     1.0   9     94.0    143.0    214.0  20.516667   

        t_max_60   t_max_90  ...  lai_v5         whc  oc_

In [18]:
# now let's write a Rds
pyreadr.write_rds("/home/germanm2/n_policy_box/Data/files_rds/prediction_set_aggregated_cnn_df.rds", prediction_set_aggregated_df)